In [1]:
import math 
import random

### Get a sieve of all numbers $n$ to indicate whether they are prime, with $0 \leq n \leq 10^8$

In [2]:
def get_primes_eratosthenes_sieve(n):
    is_prime = [1] * (n + 1)
    p = 2
    is_prime[0] = 0
    is_prime[1] = 0
    while p * p <= n:
        if is_prime[p]:
            for i in range(p * 2, n + 1, p):
                is_prime[i] = 0
        p += 1
    prime_list = []
    for i in range(n + 1):
        if is_prime[i]:
            prime_list.append(i)
    return is_prime

is_prime = get_primes_eratosthenes_sieve(10 ** 8)

### Use Pollard's Rho algorithm for finding prime factorizations for any $n$

In [3]:
# https://github.com/zhangbo2008/python_algorithm2/blob/c53669703b957a079f100c12711f86f5fc2f9389/algorithms/factorization/pollard_rho.py
def pollard_rho_prime_factorization(x):
    def f(x):
        return x * x + 1

    def rho(n, x1=2, x2=2):
        if n % 2 == 0:
            return 2
        i = 0
        while True:
            x1 = f(x1) % n
            x2 = f(f(x2)) % n
            divisor = math.gcd(abs(x1 - x2), n)
            i += 1
            if divisor != 1:
                break
            if i > 500:
                x1 = random.randint(1, 10)
                x2 = random.randint(1, 10)
                i = 0
        return divisor

    def pollard_rho_rec(x, factors):
        if x == 1:
            return

        if is_prime[x]:
            factors.append(x)
            return

        divisor = rho(x, random.randint(1, 10), random.randint(1, 10))
        pollard_rho_rec(divisor, factors)
        pollard_rho_rec(x // divisor, factors)

    if x == 1 or x == 0:
        return [(x, 1)]
    factors = []
    pollard_rho_rec(x, factors)
    return [(f, factors.count(f)) for f in set(factors)]

In [4]:
def prime_count(n, p):
    d = 0
    while n % p == 0:
        d += 1
        n /= p
    return d

In [7]:
def s(n):
    prime_factorization = pollard_rho_prime_factorization(n)
    def count_min_p_occurrences(p, exponent):
        guess = p * exponent
        total_p_multiplied, i = 0, 1
        while (i - 1) * math.log(p) - math.log(exponent) <= 0:  # p * exponent >= p ** i
            total_p_multiplied += guess // (p ** i)  # Using Legendre's formula
            i += 1
            
        # Reduce initial guess to remove the extra buffer of p factors (total_p_multiplied - exponent)
        while True:
            pc = prime_count(guess, p)
            if total_p_multiplied - pc < exponent:
                break
            guess -= p
            total_p_multiplied -= pc
            
        return guess
        
    # TODO: this is recalculating s(p^k) multiple times which can be avoided
    return max([count_min_p_occurrences(p, exponent) for p, exponent in prime_factorization])

def S(n):
    sum_s = 0
    for i in range(2, n + 1):
        sum_s += s(i)
        
        # Display progress of function execution for large values of N
        # Note: takes 1 hour 40 minutes to run for n = 10^8
        if i % (10 ** 6) == 0:
            print('Progress:', i // (10 ** 6), '%', end='\r', flush=True)
            
    return sum_s

In [8]:
S(10 ** 8)

476001479068717